# Code for full simulation

In [1]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt

## Financial Pillar Class Construction

In [2]:
class Income:
    def __init__(self, base_salary, salary_increase_rate, personal_401k_pct, employer_401k_match_pct):
        self.base_salary = base_salary
        self.salary_increase_rate = salary_increase_rate
        self.personal_401k_pct = personal_401k_pct
        self.employer_401k_match_pct = employer_401k_match_pct

    def annual_income(self, year):
        return self.base_salary * ((1 + self.salary_increase_rate / 100) ** year)

In [3]:
class Investments:
    def __init__(self, roth_ira_contrib, personal_investments, investment_growth_rate, initial_401k_balance=0):
        self.roth_ira_contrib = roth_ira_contrib
        self.personal_investments = personal_investments
        self.investment_growth_rate = investment_growth_rate / 100
        self.k_401_balance = initial_401k_balance

    def grow_investments(self):
        self.k_401_balance *= (1 + self.investment_growth_rate)
        self.personal_investments *= (1 + self.investment_growth_rate)

In [4]:
class PersonalInvestments:
    def __init__(self, balance, annual_contribution, investment_growth_rate):
        self.balance = balance  # Initial personal investment balance
        self.annual_contribution = annual_contribution  # Amount invested each year
        self.investment_growth_rate = investment_growth_rate / 100  # Convert percentage to decimal

    def contribute(self):
        """Add the annual contribution to personal investments."""
        self.balance += self.annual_contribution

    def grow(self):
        """Apply investment growth rate to the personal investments."""
        self.balance *= (1 + self.investment_growth_rate)


In [5]:
class Expenses:
    def __init__(self, food, housing, transportation, other, annual_increase_rate):
        self.food = food
        self.housing = housing
        self.transportation = transportation
        self.other = other
        self.annual_increase_rate = annual_increase_rate / 100

    def annual_expenses(self, year):
        growth_factor = (1 + self.annual_increase_rate) ** year
        return {
            'food': self.food * growth_factor,
            'housing': self.housing * growth_factor,
            'transportation': self.transportation * growth_factor,
            'other': self.other * growth_factor
        }

## Life Events

In [6]:
class LoseJob:
    def __init__(self, year, income):
        self.year = year
        self.income = income  # Store the reference to the income object

    def apply(self):
        self.income.base_salary = 0  # Now it correctly modifies the income object

In [7]:
class GainJob:
    def __init__(self, year, salary):
        self.year = year
        self.salary = salary

    def apply(self):
        self.income.base_salary = self.salary

## Build Simulation Class

In [18]:
class Simulation:
    def __init__(self, income, personal_investments, investments, expenses, losejob, years):
        self.income = income
        self.personal_investments = personal_investments
        self.investments = investments
        self.expenses = expenses
        self.years = years
        self.losejob = losejob

    def run(self):
        results = []
        for year in range(self.years):
            # Set Everything
            annual_income = self.income.annual_income(year)
            expenses_breakdown = self.expenses.annual_expenses(year)
            personal_401k = annual_income * (self.income.personal_401k_pct / 100)
            employer_401k = annual_income * (self.income.employer_401k_match_pct / 100)

            # Contribute to Retirement and Personal Investments
            self.personal_investments.contribute()

            # Grow Investments
            self.personal_investments.grow()


            self.investments.k_401_balance += personal_401k + employer_401k
            self.investments.grow_investments()

            net_cash_flow = annual_income - sum(expenses_breakdown.values()) - personal_401k

            results.append({
                'year': year + 1,
                'income': annual_income,
                'total_expenses': sum(expenses_breakdown.values()),
                'expenses': expenses_breakdown,
                'personal_investments': self.personal_investments.balance,
                'personal_401k_contrib': personal_401k,
                'employer_401k_contrib': employer_401k,
                '401k_balance': self.investments.k_401_balance,
                'roth_ira': self.investments.roth_ira_contrib,
                'personal_investments': self.personal_investments.balance,
                'cash_flow': net_cash_flow
            })

            if year == self.losejob.year:
                self.losejob.apply()


        return pd.DataFrame(results)

    def run_simulation(self):
        simulation_output = self.run()
        return simulation_output


## Run Simulation

In [19]:
def load_inputs(file_path = "inputs.yaml"):
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
        return config

In [20]:
example_yaml = load_inputs()
#example_yaml

In [23]:
def run_test():
    income = Income(**example_yaml['income'])
    personal_investments = PersonalInvestments(**example_yaml["personal_investments"])
    investments = Investments(**example_yaml['investments'])
    expenses = Expenses(**example_yaml['expenses'])
    losejob = LoseJob(example_yaml['life_events']['job_loss']['year'], income)
    sim = Simulation(income, personal_investments, investments, expenses, losejob, example_yaml['years'])
    simulation_output = sim.run_simulation()
    return simulation_output


simulation_output = run_test()
simulation_output

,year,income,total_expenses,expenses,personal_investments,personal_401k_contrib,employer_401k_contrib,401k_balance,roth_ira,cash_flow
0,1,120000.000000,43000.000000,"{'food': 8000.0, 'housing': 24000.0, 'transpor...",5350.000000,12000.000000,7200.000000,7.404400e+04,6000,65000.000000
1,2,123600.000000,44075.000000,"{'food': 8200.0, 'housing': 24599.999999999996...",11074.500000,12360.000000,7416.000000,1.003874e+05,6000,67165.000000
2,3,127308.000000,45176.875000,"{'food': 8405.0, 'housing': 25214.999999999996...",17199.715000,12730.800000,7638.480000,1.292096e+05,6000,69400.325000
3,4,131127.240000,46306.296875,"{'food': 8615.124999999996, 'housing': 25845.3...",23753.695050,13112.724000,7867.634400,1.607033e+05,6000,71708.219125
4,5,135061.057200,47463.954297,"{'food': 8830.503124999996, 'housing': 26491.5...",30766.453704,13506.105720,8103.663432,1.950750e+05,6000,74090.997183
5,6,139112.888916,48650.553154,"{'food': 9051.265703124995, 'housing': 27153.7...",38270.105463,13911.288892,8346.773335,2.325464e+05,6000,76551.046870
6,7,143286.275583,49866.816983,"{'food': 9277.54734570312, 'housing': 27832.64...",46299.012845,14328.627558,8597.176535,2.733552e+05,6000,79090.831042
7,8,147584.863851,51113.487408,"{'food': 9509.486029345699, 'housing': 28528.4...",54889.943744,14758.486385,8855.091831,3.177566e+05,6000,81712.890058
8,9,152012.409767,52391.324593,"{'food': 9747.223180079338, 'housing': 29241.6...",64082.239806,15201.240977,9120.744586,3.660241e+05,6000,84419.844197
9,10,156572.782060,53701.107708,"{'food': 9990.903759581322, 'housing': 29972.7...",73917.996593,15657.278206,9394.366924,4.184511e+05,6000,87214.396146
